Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
monitors = ["a.1", "a.2", "a.3", "a.4", "a.5", "a.6", "a.7", "bp", "bpC"],
model=m_10_04, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/scripts/10/tmp/m_10_04.stan will be updated.



Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
# Result rethinking
rethinking = "
   mean   sd  5.5% 94.5% n_eff Rhat
      mean   sd  5.5% 94.5% n_eff Rhat
a[1] -0.74 0.27 -1.17 -0.31  3838    1
a[2] 11.02 5.53  4.46 21.27  1759    1
a[3] -1.05 0.28 -1.50 -0.61  3784    1
a[4] -1.05 0.27 -1.48 -0.62  3761    1
a[5] -0.74 0.27 -1.18 -0.32  4347    1
a[6]  0.21 0.27 -0.23  0.66  3932    1
a[7]  1.81 0.39  1.19  2.46  4791    1
bp    0.84 0.26  0.42  1.26  2586    1
";

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.73587991 0.27083257 0.0042822389 0.0040916970 1000
a.2 11.00646693 5.30691251 0.0839096543 0.1342672557 1000
a.3 -1.04920876 0.28269757 0.0044698410 0.0050535050 1000
a.4 -1.04811001 0.28567652 0.0045169424 0.0047204102 1000
a.5 -0.73821690 0.27309426 0.0043179994 0.0041715074 1000
a.6  0.21728355 0.26663802 0.0042159173 0.0039551972 1000
a.7  1.81324017 0.40696017 0.0064346053 0.0068827333 1000
 bp  0.83770909 0.26524672 0.0041939188 0.0047075990 1000
bpC -0.13300269 0.29177681 0.0046133964 0.0055590719 1000

Quantiles:
       2.5%        25.0%       50.0%      75.0%       97.5%   
a.1 -1.2643628 -0.922926000 -0.7317750 -0.54974725 -0.20787122
a.2  3.8560560  6.975462500  9.9707550 13.96375000 23.95116250
a.3 -1.6152490 -1.238480000 -1.0442350 -0.85361150 -0.51094535
a.4 -1.6305108 -1.234792500 -1.0417500 -

End of `10/m10.02s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*